# 🎯 Enhanced ResNet-18 Animal Classifier
## HyperVerge Assignment - Phase 1

This notebook automatically detects the environment (Google Colab vs Local Jupyter) and adapts the setup accordingly.

**Features:**
- Environment auto-detection
- Automatic data download and setup
- Enhanced ResNet-18 with advanced training techniques
- Metal/CUDA acceleration support
- Production-ready submission format

In [1]:
# Environment Detection and Setup
import sys
import os
import subprocess
from pathlib import Path

# Detect environment
try:
    import google.colab
    IN_COLAB = True
    print("🔍 Environment: Google Colab detected")
except ImportError:
    IN_COLAB = False
    print("🔍 Environment: Local Jupyter detected")

# Set base path based on environment
if IN_COLAB:
    BASE_PATH = "/content"
else:
    BASE_PATH = "."

print(f"📁 Base path: {BASE_PATH}")

🔍 Environment: Local Jupyter detected
📁 Base path: .


In [3]:
# Install required packages based on environment
def install_packages():
    """Install required packages for both environments"""
    packages = [
        'torch', 'torchvision', 'torchaudio',
        'pandas', 'numpy', 'pillow', 'matplotlib', 'seaborn',
        'scikit-learn', 'tqdm', 'requests'
    ]
    
    if IN_COLAB:
        # Google Colab - most packages are pre-installed
        colab_packages = ['gdown']  # Only install what's missing
        for package in colab_packages:
            print(f"Installing {package}...")
            subprocess.run([sys.executable, "-m", "pip", "install", package], 
                         capture_output=True, text=True)
    else:
        # Local Jupyter - install all packages
        print("Installing packages for local environment...")
        for package in packages + ['gdown']:
            try:
                subprocess.run([sys.executable, "-m", "pip", "install", package], 
                             capture_output=True, text=True, check=True)
                print(f"✅ {package} installed")
            except subprocess.CalledProcessError:
                print(f"❌ Failed to install {package}")

install_packages()
print("📦 Package installation completed!")

Installing packages for local environment...
✅ torch installed
✅ torch installed
✅ torchvision installed
✅ torchvision installed
✅ torchaudio installed
✅ torchaudio installed
✅ pandas installed
✅ pandas installed
✅ numpy installed
✅ numpy installed
✅ pillow installed
✅ pillow installed
✅ matplotlib installed
✅ matplotlib installed
✅ seaborn installed
✅ seaborn installed
✅ scikit-learn installed
✅ scikit-learn installed
✅ tqdm installed
✅ tqdm installed
✅ requests installed
✅ requests installed
✅ gdown installed
📦 Package installation completed!
✅ gdown installed
📦 Package installation completed!


In [ ]:
# Data Download and Setup
import gdown

def download_and_setup_data():
    """Download and setup data based on environment"""
    
    if IN_COLAB:
        # Google Colab setup
        print("📥 Downloading data for Google Colab...")
        
        # Download labeled data
        gdown.download("https://drive.google.com/uc?id=18MA0qKg1rqP92HApr_Fjck7Zo4Bwdqdu", 
                      f"{BASE_PATH}/HV-AI-2025.zip", quiet=False)
        
        # Extract and organize
        os.system(f"cd {BASE_PATH} && unzip -q HV-AI-2025.zip")
        os.system(f"rm -rf {BASE_PATH}/__MACOSX")
        os.system(f"mv {BASE_PATH}/HV-AI-2025/* {BASE_PATH}/")
        os.system(f"rm -rf {BASE_PATH}/HV-AI-2025 {BASE_PATH}/HV-AI-2025.zip")
        
        # Download test data
        gdown.download("https://drive.google.com/uc?id=1aszVlQFQOwJTy9tt79s7x87VJyYw-Sxy", 
                      f"{BASE_PATH}/HV-AI-2025-Test.zip", quiet=False)
        
        # Extract test data
        os.system(f"cd {BASE_PATH} && unzip -q HV-AI-2025-Test.zip")
        os.system(f"rm -rf {BASE_PATH}/__MACOSX")
        os.system(f"mv {BASE_PATH}/HV-AI-2025-Test/* {BASE_PATH}/")
        os.system(f"rm -rf {BASE_PATH}/HV-AI-2025-Test {BASE_PATH}/HV-AI-2025-Test.zip")
        
    else:
        # Local Jupyter setup
        print("📥 Setting up data for local environment...")
        
        # Check if data already exists
        if not os.path.exists("HV-AI-2025"):
            print("Data not found locally. Please ensure HV-AI-2025 folder exists with:")
            print("  - HV-AI-2025/labeled_data/")
            print("  - HV-AI-2025/unlabeled_data/")
            print("Or manually download from the provided Google Drive links.")
        else:
            print("✅ Data folder found locally")

download_and_setup_data()

# Verify data structure
data_paths = {
    'labeled_csv': f"{BASE_PATH}/labeled_data/labeled_data.csv",
    'labeled_images': f"{BASE_PATH}/labeled_data/images",
    'unlabeled_images': f"{BASE_PATH}/unlabeled_data/images"
}

print("\n📂 Data structure verification:")
for name, path in data_paths.items():
    if os.path.exists(path):
        print(f"✅ {name}: {path}")
    else:
        print(f"❌ {name}: {path} (not found)")

In [4]:
# Import all required libraries
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset
import torchvision.transforms as transforms
import torchvision.models as models
import pandas as pd
import numpy as np
from PIL import Image
import time
import requests
from typing import Tuple, List, Dict
from tqdm import tqdm
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.utils.class_weight import compute_class_weight

# Setup device detection
def setup_device():
    """Setup computing device based on environment"""
    if IN_COLAB:
        # Google Colab - prefer CUDA if available
        if torch.cuda.is_available():
            device = torch.device("cuda")
            print("🚀 Using CUDA GPU acceleration (Google Colab)")
        else:
            device = torch.device("cpu")
            print("⚠️ Using CPU (Google Colab)")
    else:
        # Local environment - prefer Metal on macOS, then CUDA, then CPU
        if torch.backends.mps.is_available():
            device = torch.device("mps")
            print("🚀 Using Metal Performance Shaders (MPS) for GPU acceleration")
        elif torch.cuda.is_available():
            device = torch.device("cuda")
            print("🚀 Using CUDA for GPU acceleration")
        else:
            device = torch.device("cpu")
            print("⚠️ Using CPU")
    
    return device

device = setup_device()
print(f"Device: {device}")
print(f"PyTorch version: {torch.__version__}")

🚀 Using Metal Performance Shaders (MPS) for GPU acceleration
Device: mps
PyTorch version: 2.7.1


In [20]:
# Configuration Class
class Config:
    """Configuration settings adapted for both environments"""
    
    # Paths (adjusted based on environment)
    BASE_PATH = BASE_PATH
    LABELED_DATA_CSV = f"{BASE_PATH}/labeled_data/labeled_data.csv"
    LABELED_IMAGES_DIR = f"{BASE_PATH}/labeled_data/images"
    UNLABELED_IMAGES_DIR = f"{BASE_PATH}/unlabeled_data/images"
    
    # Model settings
    MODEL_NAME = "resnet18_enhanced"
    BATCH_SIZE = 32 if not IN_COLAB else 64  # Larger batch for Colab
    NUM_EPOCHS = 30 if IN_COLAB else 40    # Adjusted for environment
    LEARNING_RATE = 0.001
    WEIGHT_DECAY = 0.01
    
    # Training settings
    VALIDATION_SPLIT = 0.2
    RANDOM_SEED = 42
    EARLY_STOPPING_PATIENCE = 5
    
    # Augmentation settings
    IMAGE_SIZE = 224
    CROP_SIZE = 224
    
    # Submission settings
    EVALUATION_URL = "http://43.205.49.236:5050/inference"

print("⚙️ Configuration loaded")
print(f"📊 Batch size: {Config.BATCH_SIZE}")
print(f"🔄 Epochs: {Config.NUM_EPOCHS}")

⚙️ Configuration loaded
📊 Batch size: 32
🔄 Epochs: 40


In [21]:
# Dataset Class
class AnimalDataset(Dataset):
    """Custom Dataset class for loading animal images with labels"""
    
    def __init__(self, dataframe: pd.DataFrame, images_dir: str, transform=None):
        self.dataframe = dataframe
        self.images_dir = images_dir
        self.transform = transform
        
    def __len__(self) -> int:
        return len(self.dataframe)
    
    def __getitem__(self, idx: int) -> Tuple[torch.Tensor, int]:
        img_name = self.dataframe.iloc[idx]['img_name']
        img_path = os.path.join(self.images_dir, img_name)
        
        # Load image with error handling
        try:
            image = Image.open(img_path).convert('RGB')
        except Exception as e:
            print(f"Error loading image {img_path}: {e}")
            # Return a dummy black image if loading fails
            image = Image.new('RGB', (Config.IMAGE_SIZE, Config.IMAGE_SIZE), color='black')
        
        label = self.dataframe.iloc[idx]['encoded_label']
        
        if self.transform:
            image = self.transform(image)
            
        return image, label

print("📁 Dataset class defined")

📁 Dataset class defined


In [22]:
# Enhanced ResNet-18 Model (Strictly ResNet-18)
class EnhancedResNet18(nn.Module):
    """Enhanced ResNet-18 with improved classifier head"""
    
    def __init__(self, num_classes: int = 10, dropout_rate: float = 0.5):
        super(EnhancedResNet18, self).__init__()
        
        # Load pre-trained ResNet-18 (strictly ResNet-18)
        self.backbone = models.resnet18(weights='IMAGENET1K_V1')
        
        # Get number of features from the backbone
        num_features = self.backbone.fc.in_features
        
        # Replace the final layer with enhanced classifier
        self.backbone.fc = nn.Sequential(
            nn.Dropout(dropout_rate),
            nn.Linear(num_features, 256),
            nn.BatchNorm1d(256),
            nn.ReLU(inplace=True),
            nn.Dropout(dropout_rate * 0.6),
            nn.Linear(256, num_classes)
        )
        
    def forward(self, x: torch.Tensor) -> torch.Tensor:
        return self.backbone(x)

# Label Smoothing Loss
class LabelSmoothingCrossEntropy(nn.Module):
    """Label smoothing cross entropy loss for better generalization"""
    
    def __init__(self, epsilon: float = 0.1, weight=None):
        super().__init__()
        self.epsilon = epsilon
        self.weight = weight
        
    def forward(self, preds: torch.Tensor, target: torch.Tensor) -> torch.Tensor:
        n = preds.size()[-1]
        log_preds = torch.log_softmax(preds, dim=-1)
        loss = -log_preds.sum(dim=-1).mean()
        nll = torch.nn.functional.nll_loss(log_preds, target, weight=self.weight, reduction='mean')
        return (1 - self.epsilon) * nll + self.epsilon * loss / n

print("🏗️ Enhanced ResNet-18 model class defined")

🏗️ Enhanced ResNet-18 model class defined


In [23]:
# Data Augmentation
class DataAugmentation:
    """Advanced data augmentation strategies"""
    
    @staticmethod
    def get_train_transforms() -> transforms.Compose:
        """Enhanced training transforms"""
        return transforms.Compose([
            transforms.Resize((Config.IMAGE_SIZE + 32, Config.IMAGE_SIZE + 32)),
            transforms.RandomCrop(Config.CROP_SIZE, padding=4),
            transforms.RandomHorizontalFlip(p=0.5),
            transforms.RandomVerticalFlip(p=0.2),
            transforms.RandomRotation(degrees=20),
            transforms.RandomAffine(degrees=0, translate=(0.1, 0.1), scale=(0.9, 1.1)),
            transforms.ColorJitter(brightness=0.3, contrast=0.3, saturation=0.3, hue=0.2),
            transforms.RandomGrayscale(p=0.1),
            transforms.GaussianBlur(kernel_size=3, sigma=(0.1, 2.0)),
            transforms.ToTensor(),
            transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
            transforms.RandomErasing(p=0.1)
        ])
    
    @staticmethod
    def get_val_transforms() -> transforms.Compose:
        """Validation transforms without augmentation"""
        return transforms.Compose([
            transforms.Resize((Config.CROP_SIZE, Config.CROP_SIZE)),
            transforms.ToTensor(),
            transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
        ])
    
    @staticmethod
    def get_tta_transforms() -> List[transforms.Compose]:
        """Test Time Augmentation transforms"""
        return [
            # Original
            transforms.Compose([
                transforms.Resize((Config.CROP_SIZE, Config.CROP_SIZE)),
                transforms.ToTensor(),
                transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
            ]),
            # Horizontal flip
            transforms.Compose([
                transforms.Resize((Config.CROP_SIZE, Config.CROP_SIZE)),
                transforms.RandomHorizontalFlip(p=1.0),
                transforms.ToTensor(),
                transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
            ]),
            # Center crop
            transforms.Compose([
                transforms.Resize((Config.IMAGE_SIZE + 32, Config.IMAGE_SIZE + 32)),
                transforms.CenterCrop(Config.CROP_SIZE),
                transforms.ToTensor(),
                transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
            ])
        ]

print("🎨 Data augmentation strategies defined")

🎨 Data augmentation strategies defined


In [24]:
# Load and Prepare Data
def load_and_prepare_data():
    """Load and prepare the dataset"""
    print("📊 Loading and preparing dataset...")
    
    # Load labeled data
    df = pd.read_csv(Config.LABELED_DATA_CSV)
    
    print(f"Dataset Info:")
    print(f"Total samples: {len(df)}")
    print(f"Number of classes: {df['label'].nunique()}")
    print(f"\nClass distribution:")
    print(df['label'].value_counts())
    
    # Encode labels
    label_encoder = LabelEncoder()
    df['encoded_label'] = label_encoder.fit_transform(df['label'])
    num_classes = len(label_encoder.classes_)
    
    print(f"\nEncoded labels: {dict(zip(label_encoder.classes_, range(num_classes)))}")
    
    return df, label_encoder, num_classes

# Create Data Loaders
def create_data_loaders(df: pd.DataFrame):
    """Create train and validation data loaders"""
    print("🔄 Creating data loaders...")
    
    # Split data
    train_df, val_df = train_test_split(
        df, test_size=Config.VALIDATION_SPLIT, 
        random_state=Config.RANDOM_SEED, 
        stratify=df['label']
    )
    
    print(f"Training samples: {len(train_df)}")
    print(f"Validation samples: {len(val_df)}")
    
    # Create datasets
    train_dataset = AnimalDataset(
        train_df.reset_index(drop=True), 
        Config.LABELED_IMAGES_DIR, 
        DataAugmentation.get_train_transforms()
    )
    val_dataset = AnimalDataset(
        val_df.reset_index(drop=True), 
        Config.LABELED_IMAGES_DIR, 
        DataAugmentation.get_val_transforms()
    )
    
    # Create data loaders (adjust num_workers based on environment)
    num_workers = 0 if IN_COLAB else 0  # Use 0 for both to avoid issues
    
    train_loader = DataLoader(
        train_dataset, batch_size=Config.BATCH_SIZE, 
        shuffle=True, num_workers=num_workers, pin_memory=True
    )
    val_loader = DataLoader(
        val_dataset, batch_size=Config.BATCH_SIZE, 
        shuffle=False, num_workers=num_workers, pin_memory=True
    )
    
    return train_loader, val_loader, train_df, val_df

# Load data
df, label_encoder, num_classes = load_and_prepare_data()
train_loader, val_loader, train_df, val_df = create_data_loaders(df)

📊 Loading and preparing dataset...
Dataset Info:
Total samples: 779
Number of classes: 10

Class distribution:
label
cane          145
ragno         144
gallina        92
cavallo        78
farfalla       63
mucca          55
scoiattolo     55
pecora         54
gatto          50
elefante       43
Name: count, dtype: int64

Encoded labels: {'cane': 0, 'cavallo': 1, 'elefante': 2, 'farfalla': 3, 'gallina': 4, 'gatto': 5, 'mucca': 6, 'pecora': 7, 'ragno': 8, 'scoiattolo': 9}
🔄 Creating data loaders...
Training samples: 623
Validation samples: 156


In [25]:
# Setup Training Components
def setup_training_components(model, df, device):
    """Setup training components (loss, optimizer, scheduler)"""
    print("⚙️ Setting up training components...")
    
    # Compute class weights for imbalanced dataset
    class_weights = compute_class_weight(
        'balanced', classes=np.unique(df['encoded_label']), y=df['encoded_label']
    )
    class_weights_tensor = torch.FloatTensor(class_weights).to(device)
    
    print("Class weights:")
    for i, (cls, weight) in enumerate(zip(df['label'].unique(), class_weights)):
        count = (df['label'] == cls).sum()
        print(f"  {cls}: {count} samples (weight: {weight:.3f})")
    
    # Loss function with class weights and label smoothing
    criterion = LabelSmoothingCrossEntropy(epsilon=0.1, weight=class_weights_tensor)
    
    # Optimizer
    optimizer = optim.AdamW(
        model.parameters(), 
        lr=Config.LEARNING_RATE, 
        weight_decay=Config.WEIGHT_DECAY
    )
    
    # Learning rate scheduler
    scheduler = optim.lr_scheduler.CosineAnnealingWarmRestarts(
        optimizer, T_0=5, T_mult=2, eta_min=1e-6
    )
    
    return criterion, optimizer, scheduler

# Create and setup model
print(f"🏗️ Creating Enhanced ResNet-18 model...")
model = EnhancedResNet18(num_classes=num_classes)
model = model.to(device)

total_params = sum(p.numel() for p in model.parameters())
trainable_params = sum(p.numel() for p in model.parameters() if p.requires_grad)
print(f"Total parameters: {total_params:,}")
print(f"Trainable parameters: {trainable_params:,}")

# Setup training components
criterion, optimizer, scheduler = setup_training_components(model, df, device)

🏗️ Creating Enhanced ResNet-18 model...
Total parameters: 11,310,922
Trainable parameters: 11,310,922
⚙️ Setting up training components...
Class weights:
  cane: 145 samples (weight: 0.537)
  cavallo: 78 samples (weight: 0.999)
  elefante: 43 samples (weight: 1.812)
  farfalla: 63 samples (weight: 1.237)
  gallina: 92 samples (weight: 0.847)
  gatto: 50 samples (weight: 1.558)
  mucca: 55 samples (weight: 1.416)
  pecora: 54 samples (weight: 1.443)
  ragno: 144 samples (weight: 0.541)
  scoiattolo: 55 samples (weight: 1.416)


In [26]:
# Training Functions
def train_epoch(model, train_loader, criterion, optimizer, scheduler, device, epoch):
    """Train for one epoch"""
    model.train()
    running_loss = 0.0
    correct = 0
    total = 0
    
    progress_bar = tqdm(train_loader, desc=f"Training Epoch {epoch+1}")
    
    for batch_idx, (data, targets) in enumerate(progress_bar):
        data, targets = data.to(device), targets.to(device)
        
        # Zero gradients
        optimizer.zero_grad()
        
        # Forward pass
        outputs = model(data)
        loss = criterion(outputs, targets)
        
        # Backward pass
        loss.backward()
        
        # Gradient clipping for stability
        torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)
        
        optimizer.step()
        
        # Update learning rate within epoch for cosine annealing
        if isinstance(scheduler, optim.lr_scheduler.CosineAnnealingWarmRestarts):
            scheduler.step(epoch + batch_idx / len(train_loader))
        
        # Statistics
        running_loss += loss.item()
        _, predicted = outputs.max(1)
        total += targets.size(0)
        correct += predicted.eq(targets).sum().item()
        
        # Update progress bar
        progress_bar.set_postfix({
            'Loss': f'{running_loss/(batch_idx+1):.4f}',
            'Acc': f'{100.*correct/total:.2f}%',
            'LR': f'{optimizer.param_groups[0]["lr"]:.6f}'
        })
    
    epoch_loss = running_loss / len(train_loader)
    epoch_acc = 100. * correct / total
    
    return epoch_loss, epoch_acc

def validate_epoch(model, val_loader, criterion, device):
    """Validate for one epoch"""
    model.eval()
    running_loss = 0.0
    correct = 0
    total = 0
    
    with torch.no_grad():
        progress_bar = tqdm(val_loader, desc="Validation")
        
        for batch_idx, (data, targets) in enumerate(progress_bar):
            data, targets = data.to(device), targets.to(device)
            
            outputs = model(data)
            loss = criterion(outputs, targets)
            
            running_loss += loss.item()
            _, predicted = outputs.max(1)
            total += targets.size(0)
            correct += predicted.eq(targets).sum().item()
            
            progress_bar.set_postfix({
                'Loss': f'{running_loss/(batch_idx+1):.4f}',
                'Acc': f'{100.*correct/total:.2f}%'
            })
    
    epoch_loss = running_loss / len(val_loader)
    epoch_acc = 100. * correct / total
    
    return epoch_loss, epoch_acc

print("🔧 Training functions defined")

🔧 Training functions defined


In [27]:
# Main Training Loop
def train_model():
    """Complete training loop"""
    print(f"🚀 Starting Enhanced ResNet-18 Training for {Config.NUM_EPOCHS} epochs...")
    print(f"🖥️  Environment: {'Google Colab' if IN_COLAB else 'Local Jupyter'}")
    print(f"🎯 Device: {device}")
    
    # Training history
    history = {
        'train_loss': [], 'train_acc': [],
        'val_loss': [], 'val_acc': []
    }
    
    best_val_acc = 0.0
    patience_counter = 0
    start_time = time.time()
    
    for epoch in range(Config.NUM_EPOCHS):
        print(f"\n🔥 Epoch {epoch+1}/{Config.NUM_EPOCHS}")
        print("-" * 60)
        
        # Train
        train_loss, train_acc = train_epoch(
            model, train_loader, criterion, optimizer, scheduler, device, epoch
        )
        
        # Validate
        val_loss, val_acc = validate_epoch(model, val_loader, criterion, device)
        
        # Update scheduler (for non-cosine schedulers)
        if not isinstance(scheduler, optim.lr_scheduler.CosineAnnealingWarmRestarts):
            scheduler.step()
        
        # Store metrics
        history['train_loss'].append(train_loss)
        history['train_acc'].append(train_acc)
        history['val_loss'].append(val_loss)
        history['val_acc'].append(val_acc)
        
        print(f"📊 Train Loss: {train_loss:.4f}, Train Acc: {train_acc:.2f}%")
        print(f"📊 Val Loss: {val_loss:.4f}, Val Acc: {val_acc:.2f}%")
        print(f"📊 Learning Rate: {optimizer.param_groups[0]['lr']:.6f}")
        
        # Save best model
        if val_acc > best_val_acc:
            best_val_acc = val_acc
            
            # Save checkpoint
            torch.save({
                'epoch': epoch,
                'model_state_dict': model.state_dict(),
                'optimizer_state_dict': optimizer.state_dict(),
                'best_val_acc': best_val_acc,
                'label_encoder': label_encoder,
                'history': history
            }, f'{Config.BASE_PATH}/best_enhanced_resnet18.pth')
            
            print(f"🎯 NEW BEST! Model saved with validation accuracy: {val_acc:.2f}%")
            patience_counter = 0
        else:
            patience_counter += 1
        
        # Early stopping
        if patience_counter >= Config.EARLY_STOPPING_PATIENCE:
            print(f"📈 Early stopping triggered after {epoch+1} epochs")
            break
    
    total_time = time.time() - start_time
    print(f"\n✅ Training completed in {total_time/60:.1f} minutes")
    print(f"🏆 Best validation accuracy: {best_val_acc:.2f}%")
    
    return history, best_val_acc

# Start training
history, best_val_acc = train_model()

🚀 Starting Enhanced ResNet-18 Training for 40 epochs...
🖥️  Environment: Local Jupyter
🎯 Device: mps

🔥 Epoch 1/40
------------------------------------------------------------


Training Epoch 1:   0%|          | 0/20 [00:00<?, ?it/s]/Users/hariharan/Hiring/HyperVerge/venv/lib/python3.13/site-packages/torch/utils/data/dataloader.py:683: UserWarning: 'pin_memory' argument is set as true but not supported on MPS now, then device pinned memory won't be used.
  warnings.warn(warn_msg)
/Users/hariharan/Hiring/HyperVerge/venv/lib/python3.13/site-packages/torch/utils/data/dataloader.py:683: UserWarning: 'pin_memory' argument is set as true but not supported on MPS now, then device pinned memory won't be used.
  warnings.warn(warn_msg)
Validation: 100%|██████████| 5/5 [00:00<00:00, 14.20it/s, Loss=6.1566, Acc=16.03%]



📊 Train Loss: 2.0893, Train Acc: 26.32%
📊 Val Loss: 6.1566, Val Acc: 16.03%
📊 Learning Rate: 0.000914
🎯 NEW BEST! Model saved with validation accuracy: 16.03%

🔥 Epoch 2/40
------------------------------------------------------------


Validation: 100%|██████████| 5/5 [00:00<00:00, 15.48it/s, Loss=2.1413, Acc=33.33%]


📊 Train Loss: 1.8809, Train Acc: 38.20%
📊 Val Loss: 2.1413, Val Acc: 33.33%
📊 Learning Rate: 0.000670
🎯 NEW BEST! Model saved with validation accuracy: 33.33%

🔥 Epoch 3/40
------------------------------------------------------------


...

📊 Train Loss: 0.9019, Train Acc: 82.34%
📊 Val Loss: 1.3419, Val Acc: 65.38%
📊 Learning Rate: 0.000001

🔥 Epoch 16/40
------------------------------------------------------------


Validation: 100%|██████████| 5/5 [00:00<00:00, 14.90it/s, Loss=3.0313, Acc=28.85%]


📊 Train Loss: 1.1314, Train Acc: 73.68%
📊 Val Loss: 3.0313, Val Acc: 28.85%
📊 Learning Rate: 0.000994

🔥 Epoch 17/40
------------------------------------------------------------


Validation: 100%|██████████| 5/5 [00:00<00:00, 14.76it/s, Loss=2.2211, Acc=44.23%]


📊 Train Loss: 1.3535, Train Acc: 62.60%
📊 Val Loss: 2.2211, Val Acc: 44.23%
📊 Learning Rate: 0.000977
📈 Early stopping triggered after 17 epochs

✅ Training completed in 1.5 minutes
🏆 Best validation accuracy: 67.31%


In [ ]:
# Visualization
def plot_training_history(history):
    """Plot training history"""
    fig, axes = plt.subplots(1, 2, figsize=(15, 5))
    
    # Loss plot
    axes[0].plot(history['train_loss'], label='Train Loss', color='blue')
    axes[0].plot(history['val_loss'], label='Validation Loss', color='red')
    axes[0].set_title('Training and Validation Loss')
    axes[0].set_xlabel('Epoch')
    axes[0].set_ylabel('Loss')
    axes[0].legend()
    axes[0].grid(True)
    
    # Accuracy plot
    axes[1].plot(history['train_acc'], label='Train Accuracy', color='blue')
    axes[1].plot(history['val_acc'], label='Validation Accuracy', color='red')
    axes[1].set_title('Training and Validation Accuracy')
    axes[1].set_xlabel('Epoch')
    axes[1].set_ylabel('Accuracy (%)')
    axes[1].legend()
    axes[1].grid(True)
    
    plt.tight_layout()
    plt.show()

# Plot results
print("📊 Generating training visualizations...")
plot_training_history(history)

# Print final results
print(f"\n🎯 FINAL RESULTS:")
print(f"✅ Best Validation Accuracy: {best_val_acc:.2f}%")
print(f"📁 Model saved as: best_enhanced_resnet18.pth")
print(f"🏗️ Architecture: Enhanced ResNet-18")
print(f"🖥️  Environment: {'Google Colab' if IN_COLAB else 'Local Jupyter'}")

In [ ]:
# Inference and Submission
class ModelInference:
    """Model inference with Test Time Augmentation"""
    
    def __init__(self, model, device):
        self.model = model
        self.device = device
        self.tta_transforms = DataAugmentation.get_tta_transforms()
    
    def predict_single_image(self, image_path, label_encoder, use_tta=True):
        """Predict single image with optional TTA"""
        self.model.eval()
        
        try:
            image = Image.open(image_path).convert('RGB')
        except Exception as e:
            print(f"Error loading image {image_path}: {e}")
            return "unknown", 0.0
        
        if use_tta:
            predictions = []
            with torch.no_grad():
                for transform in self.tta_transforms:
                    image_tensor = transform(image).unsqueeze(0).to(self.device)
                    outputs = self.model(image_tensor)
                    probabilities = torch.nn.functional.softmax(outputs, dim=1)
                    predictions.append(probabilities.cpu().numpy())
            
            # Average predictions
            avg_predictions = np.mean(predictions, axis=0)
            predicted_class_idx = np.argmax(avg_predictions)
            confidence = avg_predictions[0][predicted_class_idx]
        else:
            transform = DataAugmentation.get_val_transforms()
            image_tensor = transform(image).unsqueeze(0).to(self.device)
            
            with torch.no_grad():
                outputs = self.model(image_tensor)
                probabilities = torch.nn.functional.softmax(outputs, dim=1)
                predicted_class_idx = torch.argmax(probabilities, dim=1).item()
                confidence = probabilities.max().item()
        
        predicted_class = label_encoder.inverse_transform([predicted_class_idx])[0]
        return predicted_class, confidence
    
    def generate_submission(self, test_images_dir, label_encoder, output_csv='phase1_predictions.csv', use_tta=True):
        """Generate submission file in required format"""
        from pathlib import Path
        
        # Get all test image files
        test_images = []
        test_dir_path = Path(test_images_dir)
        
        if test_dir_path.exists():
            for ext in ['*.jpg', '*.jpeg', '*.png']:
                test_images.extend(test_dir_path.glob(ext))
        else:
            print(f"❌ Test directory not found: {test_images_dir}")
            return None
        
        if len(test_images) == 0:
            print(f"❌ No test images found in: {test_images_dir}")
            return None
        
        predictions = []
        
        print(f"🔍 Generating predictions for {len(test_images)} test images...")
        print(f"🎯 Using TTA: {use_tta}")
        
        for img_path in tqdm(test_images, desc="Predicting"):
            predicted_class, confidence = self.predict_single_image(
                str(img_path), label_encoder, use_tta
            )
            
            predictions.append({
                'path': img_path.name,  # Just filename as required
                'predicted_label': predicted_class
            })
        
        # Create DataFrame and save
        pred_df = pd.DataFrame(predictions)
        pred_df.to_csv(f"{Config.BASE_PATH}/{output_csv}", index=False)
        
        print(f"✅ Predictions saved to {output_csv}")
        print(f"📊 Format: path,predicted_label")
        print(f"📊 Total predictions: {len(predictions)}")
        
        # Show statistics
        print(f"\n📋 Sample predictions:")
        print(pred_df.head(10))
        
        print(f"\n📈 Predicted class distribution:")
        print(pred_df['predicted_label'].value_counts())
        
        return pred_df

# Setup inference
inference = ModelInference(model, device)

print("🔍 Inference pipeline ready!")
print("\n📋 To generate predictions on test data:")
print("1. Ensure test images are available")
print("2. Run the prediction code below")

In [ ]:
# Generate Predictions (Example)
# Uncomment and modify the following code when you have test data

"""
# Example: Generate predictions for test data
test_images_dir = f"{Config.BASE_PATH}/test_data/images"  # Update path as needed

# Check if test directory exists
if os.path.exists(test_images_dir):
    print(f"📁 Test directory found: {test_images_dir}")
    
    # Generate predictions
    predictions_df = inference.generate_submission(
        test_images_dir=test_images_dir,
        label_encoder=label_encoder,
        output_csv='phase1_predictions.csv',
        use_tta=True
    )
    
    if predictions_df is not None:
        print("🎯 Predictions generated successfully!")
        
        # Show submission format
        print("\\n📤 Submission file format:")
        print(predictions_df.head())
else:
    print(f"❌ Test directory not found: {test_images_dir}")
    print("Please update the test_images_dir path or ensure test data is available")
"""

print("💡 Uncomment the code above to generate predictions when test data is available")

In [ ]:
# Result Submission Helper
def send_results_for_evaluation(name, csv_file, email):
    """Send results to evaluation server"""
    try:
        url = "http://43.205.49.236:5050/inference"
        files = {'file': open(f"{Config.BASE_PATH}/{csv_file}", 'rb')}
        data = {'email': email, 'name': name}
        response = requests.post(url, files=files, data=data)
        return response.json()
    except Exception as e:
        print(f"Error submitting results: {e}")
        return {"error": str(e)}

# Example submission (uncomment when ready)
"""
# Submit results to evaluation server
result = send_results_for_evaluation(
    name="Your Name",
    csv_file="phase1_predictions.csv",
    email="your.email@example.com"
)
print(f"Submission result: {result}")
"""

print("📤 Submission helper ready!")
print("Update name and email in the code above, then uncomment to submit results")

## 🏆 Training Complete!

### Results Summary:
- **Architecture**: Enhanced ResNet-18 (strictly ResNet-18 backbone)
- **Environment**: Auto-detected and optimized
- **Best Validation Accuracy**: See output above
- **Model Saved**: `best_enhanced_resnet18.pth`

### Key Features Implemented:
1. ✅ **Environment Auto-Detection** (Colab vs Local)
2. ✅ **Automatic Data Download** (Colab) / Local Setup
3. ✅ **Enhanced ResNet-18** with improved classifier
4. ✅ **Advanced Data Augmentation** (10+ techniques)
5. ✅ **Class Weight Balancing** for imbalanced dataset
6. ✅ **Label Smoothing** for better generalization
7. ✅ **Test Time Augmentation** for inference
8. ✅ **Early Stopping** to prevent overfitting
9. ✅ **Submission Format** ready for evaluation

### Next Steps:
1. **Test Data**: Ensure test images are available
2. **Predictions**: Uncomment prediction code and run
3. **Submit**: Update email/name and submit to evaluation server

### Phase 2 Preparation:
This ResNet-18 model can serve as the foundation for Phase 2 semi-supervised learning with unlabeled data.